In [ ]:
import tensorflow as tf
from functools import partial
from libs.pipeline_utils import build_sampler_env, sample
from libs.pipeline_utils import wrap_list_with_score, simple_probability_pipeline, probability
from libs.utils import load_dictionary, load_transformer, top_best, load_dis

In [ ]:
n_items = 1000
batch_size = 64
enc_seq_len = 64
dec_seq_len = 64

In [ ]:
load_dir = "./models/shm_c1"
sampler_args = build_sampler_env(load_dir, batch_size, enc_seq_len, dec_seq_len)

In [ ]:
dictionary = load_dictionary("./data/")
transformer = load_transformer(load_dir)

In [ ]:
with tf.device("/cpu:0"):
    dis_be = load_dis('./models/', 'believability')
    dis_st = load_dis('./models/', 'style')

In [ ]:
be_f = partial(probability, model=dis_be, transformer=transformer)
st_f = partial(probability, model=dis_st, transformer=transformer)

In [ ]:
sample_f = partial(sample, *sampler_args, 
                   dictionary=dictionary, 
                   transformer=transformer, 
                   n_items=n_items,
                   batch_size=64)

In [ ]:
seed_phrase = 'Прогуляемся по крышам'
simple_probability_pipeline(seed_phrase, sample_f, [be_f, st_f], topn=0.02)